In [2]:
import json
import requests
import time
import pandas as pd

# Get list of ids of all games 

    # Opening JSON file
f = open('apps.json','rb')
 
    # returns JSON object as
    # a dictionary
data = json.load(f)
 
    # Iterating through the json
    # list
list01=[]
for i in data:
    list01.append(i['appID'])
 
    # Closing file
f.close()

In [4]:
# Use the list of ids to get all data from the API

def request_steam_data(list_input):
    list_res = []
    for i in list_input:
        responses = requests.get('https://api.steamapis.com/market/app/'+ str(i) +'?api_key=zl4MTjkmTqiqajCFXBxcATIuZUY')
        code = responses.status_code
        if code < 300:
            list_res.append(responses.json())
        else:
            raise ValueError(f'Response error with code {code}.')
    return list_res

data = request_steam_data(list01)

In [5]:
# Process data


df = pd.DataFrame(data)[['appID','name','release_date','price_overview','developers','categories','genres','recommendations']]

    # get information in release_date
df['date'] = [df['release_date'][i]['date'] for i in df.index]
df['coming soon'] = [df['release_date'][i]['coming_soon'] for i in df.index]


    # get prices from price_overview
def get_price(index):
    list_res=[]
    for i in range(0,df['appID'].count()):
        if data[i].get('price_overview')!=None:
            list_res.append(int(str(data[i]['price_overview']).split(': ')[index].split(', ')[0])/100)
        else:
            list_res.append(0)
    return pd.Series(list_res)
df['initial price'] = get_price(-3)
df['final price'] = get_price(-2)


    # get developers
list_dev = []
for i in range(0,df['appID'].count()):
    if data[i].get('developers') == None:
        list_dev.append('None')
    elif len(data[i].get('developers')) > 1:
        list_dev.append(', '.join(data[i].get('developers')))
    else:
        list_dev.append(data[i].get('developers')[0])
df['dev'] = pd.Series(list_dev)


    # get category
list_cate = []
for i in range(0,df['appID'].count()):
    if data[i].get('categories') == None:
        list_cate.append('None')
    else:
        list_res = []
        for j in range(0, len(data[i].get('categories'))):
            list_res.append(data[i].get('categories')[j]['description'])
        list_cate.append(', '.join(list_res))
df['category'] = pd.Series(list_cate)


    # get genre
list_genre = []
for i in range(0,df['appID'].count()):
    if data[i].get('genres') == None:
        list_genre.append('None')
    else:
        list_res = []
        for j in range(0, len(data[i].get('genres'))):
            list_res.append(data[i].get('genres')[j]['description'])
        list_genre.append(', '.join(list_res))
df['genre'] = pd.Series(list_genre)


    # get recommendations
list_recomm = []
for i in range(0,df['appID'].count()):
    if data[i].get('recommendations') == None:
        list_recomm.append(0)
    else:
        list_recomm.append(data[i].get('recommendations')['total'])
df['recomm'] = pd.Series(list_recomm)

df_final = df[['appID','name','date','coming soon','initial price','final price','dev','category','genre','recomm']]\
           .rename(columns={'appID':'Appid','name':'Game name','date':'Release date','coming soon':'Coming soon','initial price':'Initial price',\
           'final price':'Final price','dev':'Developer','category':'Category','genre':'Genre','recomm':'Recommendations'})
df_final

,Appid,Game name,Release date,Coming soon,Initial price,Final price,Developer,Category,Genre,Recommendations
0,250700,Super Time Force Ultra,"Aug 25, 2014",False,14.99,14.99,Capybara Games,"Single-player, Steam Achievements, Full contro...","Action, Indie",571
1,531490,Golden Swords,"Oct 26, 2016",False,3.99,3.99,Vlad K.,"Single-player, Steam Trading Cards","Adventure, Indie, RPG, Strategy",0
2,10500,Total War: EMPIRE – Definitive Edition,"Mar 4, 2009",False,24.99,24.99,"CREATIVE ASSEMBLY, Feral Interactive (Mac), Fe...","Single-player, Multi-player, PvP, Online PvP, ...",Strategy,11335
3,454930,Under Zero,"Mar 2, 2017",False,3.99,3.99,Dagestan Technology,"Single-player, Steam Achievements, Steam Tradi...","Action, Adventure, Indie, RPG",746
4,423900,Dinocide,"Jan 21, 2016",False,9.99,9.99,AtomicTorch Studio,"Single-player, Full controller support, Steam ...","Action, Adventure, Casual, Indie, RPG",179
...,...,...,...,...,...,...,...,...,...,...
10205,940710,Colt Canyon,"Jun 16, 2020",False,14.99,14.99,Retrific,"Single-player, Multi-player, Co-op, Shared/Spl...","Action, Casual, Indie, RPG",131
10206,958400,Project CARS 3,"Aug 27, 2020",False,59.99,59.99,Slightly Mad Studios,"Single-player, Multi-player, PvP, Online PvP, ...",Racing,659
10207,557420,Kamio Recoil,"Dec 21, 2016",False,7.99,7.99,Shindenken,Single-player,"Action, Indie",0
10208,951430,Dead Age 2,"Jul 16, 2020",False,13.49,17.99,Silent Dreams,"Single-player, Steam Achievements, Full contro...","Action, Adventure, Casual, Indie, RPG, Simulat...",269


In [7]:
# Output data as .csv files

df.to_csv('raw_data.csv')
df_final.to_csv('processed_data.csv')